In [ ]:
import pandas as pd
from tqdm import tqdm
import torch

/home/azureuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import pandas as pd
import difflib
from tqdm import tqdm
 
# Load all Excel files
query_df = pd.read_excel("query_data_ready.xlsx")
inboedel_df = pd.read_excel("Groundtruth_inboedel_enriched.xlsx")
reis_df = pd.read_excel("Groundtruth_reis_enriched.xlsx")
gt_df = pd.concat([inboedel_df, reis_df], ignore_index=True)
 
# Normalize vragen
query_df["question"] = query_df["question"].astype(str).str.strip().str.lower()
gt_df["vraag"] = gt_df["vraag"].astype(str).str.strip().str.lower()
 
# Function: best GT match
def find_best_match(query_vraag):
    matches = difflib.get_close_matches(query_vraag, gt_df["vraag"], n=1, cutoff=0.6)
    if not matches:
        return pd.Series([None, 0.0, []])
    matched = matches[0]
    score = difflib.SequenceMatcher(None, query_vraag, matched).ratio()
    sources = gt_df[gt_df["vraag"] == matched]["source"].unique().tolist()
    return pd.Series([matched, score, sources])
 
# Apply to all rows
tqdm.pandas()
query_df[["matched_gt_vraag", "similarity_score", "matched_sources"]] = query_df["vraag"].progress_apply(find_best_match)
 
# Save result
query_df.to_excel("vraag_matched_output.xlsx", index=False)
print("Saved as vraag_matched_output.xlsx")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 1370.97it/s]

Saved as vraag_matched_output.xlsx


In [2]:
# 1. Load input Excel files
query_df = pd.read_excel("query_data_ready.xlsx")
inboedel_df = pd.read_excel("Groundtruth_inboedel_enriched.xlsx")
reis_df = pd.read_excel("Groundtruth_reis_enriched.xlsx")
gt_df = pd.concat([inboedel_df, reis_df], ignore_index=True)
 

In [3]:
import openai

import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(dotenv_path=".env")


True

In [4]:
load_dotenv()
 
client = OpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("ENDPOINT_URL") + "/openai/deployments/" + os.getenv("DEPLOYMENT_NAME_EMBEDDING"),
    default_headers={"api-key": os.getenv("AZURE_OPENAI_API_KEY")},
)
 
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="",  
        encoding_format="float"
    )
    return np.array(response.data[0].embedding)

In [5]:
# 3. Source parsing functions
def parse_inboedel_source(source: str):
    source = source.lower()
    parts = source.split("_")
    if "aegon" in parts:
        polis_versie = "Oud (3038)" if "oud" in parts else "Nieuw (3041)"
        dekking = "Allrisk" if "allrisk" in parts else "Basis"
        return {
            "product": "Inboedel",
            "polis_versie": polis_versie,
            "type_klant": None,
            "dekking": dekking
        }
    elif "a.s.r." in source:
        type_klant = "Adviseur" if "advies" in parts else "Direct"
        dekking = "Allrisk" if "topdekking" in source or "allrisk" in parts else "Basis"
        return {
            "product": "Inboedel",
            "polis_versie": None,
            "type_klant": type_klant,
            "dekking": dekking
        }
    return None
 
def parse_reis_source(source: str):
    source = source.lower()
    if "aegon_basis" in source:
        return {"product": "Doorlopende Reisverzekering", "polis_versie": None, "type_klant": None, "dekking": "Basis"}
    elif "aegon_allrisk" in source:
        return {"product": "Doorlopende Reisverzekering", "polis_versie": None, "type_klant": None, "dekking": "Allrisk"}
    elif "a.s.r._vp_dr_2024" in source:
        return {"product": "Doorlopende Reisverzekering", "polis_versie": None, "type_klant": "Adviseur", "dekking": "Basis"}
    elif "ik_kies_zelf_(dr_2018)" in source:
        return {"product": "Doorlopende Reisverzekering", "polis_versie": None, "type_klant": "Direct", "dekking": "Allrisk"}
    return None

In [6]:
# 4. Parse sources
gt_df["parsed_meta"] = gt_df["source"].apply(
    lambda s: parse_inboedel_source(s) if "inboedel" in str(s).lower() else parse_reis_source(s)
)
 

In [7]:
# 5. matching logic
from sklearn.metrics.pairwise import cosine_similarity

def find_sources_for_best_match(query_vraag):
    if not isinstance(query_vraag, str):
        return pd.Series([None]*4)
 
    query_embedding = get_embedding(query_vraag)
    all_gt_vragen = gt_df["vraag"].astype(str).tolist()
    all_gt_embeddings = np.array([get_embedding(v) for v in all_gt_vragen])

    scores = cosine_similarity([query_embedding], all_gt_embeddings)[0]
    best_idx = int(np.argmax(scores))
    best_score = float(scores[best_idx])
    matched_vraag = all_gt_vragen[best_idx]
 
    matched_rows = gt_df[gt_df["vraag"] == matched_vraag].copy()
    all_sources = matched_rows["source"].unique().tolist()
    mapped_sources = matched_rows["parsed_meta"].tolist()
 
    return pd.Series([
        matched_vraag,
        all_sources,
        mapped_sources,
        best_score
    ])

In [8]:
# 6. Apply to query set
tqdm.pandas()
query_df[[
    "matched_gt_vraag",
    "all_sources",
    "mapped_sources",
    "similarity_score"
]] = query_df.progress_apply(lambda row: find_sources_for_best_match(row["vraag"]), axis=1)

  3%|█████████████████████████████▏                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 1/33 [00:00<00:12,  2.51it/s]


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [ ]:
# 7. Save output
query_df.to_excel("query_matched_sources_output.xlsx", index=False)
print("Matching completed and saved to query_matched_sources_output.xlsx")